# Count all animals in the JSON files
You can chose if you want to have it per camera or not

In [ ]:
import json
import csv
import os
import re

annotation_location = os.path.join("stats") # Don't forget to put the path to the folder here!

def filter_images(json_folder, per_camera=False):
    animal_dict = {} # To memorize all animals that we had
    animal_id_dict = {} # Dict with the id per animal
    unknown_dict = {}
    for root, _, files in os.walk(json_folder):
        for file in files:
            if not file.endswith('.json'):
                continue
            camera = None
            try:
                camera = re.findall(r'((?:awc|gkz)\d{2})', file, flags=re.IGNORECASE)[0].upper()
            except:
                print(f'No camera found for -> {file}')
                camera = 'UNK' # UNK is an abbreviation for unknown
                if camera in unknown_dict.keys():
                    unknown_dict[camera] += 1
                else:
                    unknown_dict[camera] = 0
                camera = f'{camera}_{unknown_dict[camera]}'
            print(f'Camera -> {camera}')

            temp_dict = {}
            data = None
            path = os.path.join(root, file)
            print(f'Processing -> {path}')

            # Read JSON file
            with open(path, 'r') as f:
                data = json.load(f)

            # Add animals, missing to the animal and animal id dictionary
            catagory_data = data['categories']
            skip_key = None # We don't want to count the miscellaneous category!
            for item in catagory_data:
                animal = re.sub(r'\s+', '_' , item['name'])
                id = item['id']
                if skip_key == None and animal == 'miscellaneous':
                    skip_key = id
                    print(f'Skip key is: {skip_key}')

                if animal not in animal_dict.keys() and animal != 'miscellaneous':
                    if per_camera:
                        temp_dict[animal] = 0
                    else:
                        animal_dict[animal] = 0
                    animal_id_dict[id] = animal
            
            image_data = data['annotations']
            for item in image_data:
                if item['category_id'] != skip_key:
                    if per_camera:
                        temp_dict[animal_id_dict[item['category_id']]] += 1
                    else:
                        animal_dict[animal_id_dict[item['category_id']]] += 1
            

            if per_camera and (camera not in list(animal_dict.keys())):
                animal_dict[camera] = temp_dict
            elif per_camera:
                keys = list(temp_dict.keys())
                for key in keys:
                    if key in list(animal_dict[camera].keys()):
                        animal_dict[camera][key] += temp_dict[key]
                    else:
                        animal_dict[camera][key] = temp_dict[key]

    return animal_dict

## All animals, not per camera

In [ ]:
print(annotation_location)
result = filter_images(annotation_location)
key_list = result.keys()

result_list = []

print(result)

for key in key_list:
    result_list.append((key, result[key]))
    print(f'{key} -> {result[key]}')

In [ ]:
output_location = os.path.join(annotation_location, "annotation_result.csv")
header = ["Animal", "Count"]

if os.path.exists(output_location):
    os.remove(output_location)
    print(f"{output_location} has been removed.")
else:
    print(f"{output_location} does not exist. Proceeding to create a new file.")

with open(output_location, 'w', newline='') as file:
     writer = csv.writer(file)
     writer.writerow(header)
     writer.writerows(result_list)

## All animals, per camera

In [ ]:
result = filter_images(annotation_location, per_camera=True)

camera_list = result.keys()

result_list = []

for camera in camera_list:
    temp_results = [camera]
    key_list = result[camera].keys()
    for key in key_list:
        temp_results.append((key, result[camera][key]))
        print(f'{camera}\n\t{key} -> {result[camera][key]}')
    result_list.append((temp_results))

In [ ]:
output_location = os.path.join(annotation_location, f"per_camera_annotation_result.csv")
header = ["Camera", "Animal", "Count"]

if os.path.exists(output_location):
    os.remove(output_location)
    print(f"{output_location} has been removed.")
else:
    print(f"{output_location} does not exist. Proceeding to create a new file.")

with open(output_location, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

for result in result_list:
    camera = result[0]
    result_data = result[1:]
    final_data = [(camera, *t) for t in result_data]
    with open(output_location, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(final_data)

# Get the animal stats

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
all_data = pd.read_csv(os.path.join(annotation_location, f"annotation_result.csv"), sep=',')
per_data = pd.read_csv(os.path.join(annotation_location, f"per_camera_annotation_result.csv"), sep=',')

In [ ]:
sorted_all_data = all_data.sort_values('Count', axis=0, ascending=False)

# Functions

In [ ]:
import numpy as np
def prepare_data_for_pie(df):
    total_value = df['Count'].sum()

    # Compute the percentage for each category
    df['Percentage'] = (df['Count'] / total_value) * 100

    # Separate categories into two groups: "rest" and others
    rest = df[df['Percentage'] <= 2].copy()  # Categories below 1.5%
    others = df[df['Percentage'] > 2].copy()  # Categories above or equal to 1.5%

    # Aggregate "rest" categories into a single entry
    if not rest.empty:
        rest_value = rest['Count'].sum()
        rest_category = pd.DataFrame({'Animal': [f'rest'], 'Count': [rest_value]})
        final_df = pd.concat([others, rest_category], ignore_index=True)
    else:
        final_df = others  # No "rest" category needed if all values >= 1.5%

    return final_df

def make_pie_chart(df, title, file_name):
    df = df.copy()
    df = prepare_data_for_pie(df)
    
    # Plot the pie chart
    plt.figure(figsize=(8, 8))
    wedges, texts, autotexts = plt.pie(
        df['Count'], 
        labels=df['Animal'], 
        autopct='%1.1f%%', 
        startangle=90, 
        pctdistance=0.85, 
        explode=[0.1] * len(df)
    )

    plt.title(title)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.savefig(os.path.join('results', file_name), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def make_list(df, file_name):
    # Create a matplotlib figure and axis
    int(df.shape[0]/6)+1
    fig, ax = plt.subplots(figsize=(7, round(df.shape[0]/6)))  # Adjust size as needed
    ax.axis('tight')  # Remove axes
    ax.axis('off')    # Hide axes

    # Create the table in the axis
    try:
        table = ax.table(
            cellText=df.values,
            colLabels=df.columns,
            cellLoc='center',
            loc='center'
        )
    except IndexError:
        return

    # Alternate row colors
    colors = ['lightgrey', 'darkgrey']  # Define alternating colors
    for (row, col), cell in table.get_celld().items():
        if row == 0:  # Header row
            cell.set_text_props(weight='bold')  # Make headers bold
            cell.set_facecolor('grey')         # Set header background color
        else:
            cell.set_facecolor(colors[(row - 1) % 2])  # Alternate row colors

    # Adjust the layout to ensure the title doesn't overlap with the table
    plt.subplots_adjust(top=0.85)  # Adjust space between the title and the table

    plt.savefig(os.path.join('results', file_name), dpi=300, bbox_inches='tight')
    # Display the table
    plt.show()

In [ ]:
def make_horizontal_bar(df, file_name, multiple=False, camera=None, y_label='Animal'):
    if not multiple:
        plt.figure(figsize=(12.5, 10))
        plt.barh(df[y_label], df['Count'], color='orange')

        if camera is None:
            plt.title("All animals counted on all camera's\n- Count more than 0")
        elif y_label == 'Camera':
            plt.title == f"All {camera} on camera's"
        else:
            plt.title(f"All animals counted on camera: {camera}\n- Count more than 0")
        plt.xlabel('Count', fontsize=12)
        plt.ylabel(y_label, fontsize=12)
        plt.savefig(os.path.join('results', file_name), dpi=300, bbox_inches='tight')
        plt.show()
    else:
        cameras = list(set(df['Camera']))
        for camera in cameras:
            temp_df = df[df['Camera'] == camera]
            temp_non_df = temp_df[temp_df['Count'] == 0]
            temp_df = temp_df[temp_df['Count'] > 0]

            make_horizontal_bar(temp_df, f'hbar_{camera}.png', camera=camera)
            make_list(temp_df, f'spotted_{camera}.png')
            make_list(temp_non_df, f'not_spotted_{camera}.png')
            make_pie_chart(temp_df, f'Pie chart of all animals on camera {camera}', f'pie_{camera}.png')

In [ ]:
def make_bar(df, file_name, camera=None, y_label='Animal', x_label='Count'):
    plt.figure(figsize=(7, 5))
    plt.bar(df[x_label], df[y_label], color='orange')

    if camera is None:
        plt.title("All animals counted on all camera's\n- Count more than 0")
    elif y_label == 'Count':
        plt.title(f"All {camera} on camera's")
    else:
        plt.title(f"All animals counted on camera: {camera}\n- Count more than 0")
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.savefig(os.path.join('results', file_name), dpi=300, bbox_inches='tight')
    plt.show()

# All animals

In [ ]:
make_horizontal_bar(sorted_all_data[sorted_all_data['Count'] > 0], 'hbar_all.png')

### All spoted animals

In [ ]:
spotted = sorted_all_data[sorted_all_data['Count'] > 0]

In [ ]:
make_list(spotted, 'spotted_all.png')

### All not spoted animals

In [ ]:
not_spotted = sorted_all_data[sorted_all_data['Count'] == 0]

In [ ]:
make_list(not_spotted, 'not_spotted_all.png')

In [ ]:
make_pie_chart(spotted, 'Pie chart of all animals', 'pie_all.png')